In [1]:
from datasets import load_dataset
import random
import re
from tqdm import tqdm
import pickle
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
random.seed(42)


/home/ardrit/MLX/week_2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
marco_dataset = load_dataset('ms_marco', 'v1.1')

In [3]:
with open('tokenizer.pickle', 'rb') as file:
    tokenizer = pickle.load(file)

In [4]:
def tokenize(word, tokenizer=tokenizer):
    if word in tokenizer.keys():
        return tokenizer[word]
    else:
        return 0


In [5]:
inverse_tokens = {}
def inverse(tokenizer):
    for token in tokenizer:

        inverse_tokens[tokenizer[token]] = token
inverse(tokenizer)

In [6]:
def reverse_tokenize(word,inverse_tokens=inverse_tokens):
    if (int(word) == 0):
        return None
    return inverse_tokens[word]

In [7]:
tokenized_marco_text = []
for split in marco_dataset:
    tokenized_sample = []
    for sample in marco_dataset[split]:
        for words in (sample['passages']['passage_text']):
            words_ = re.sub(r'[^a-zA-Z-\s]', '', words)
            words_ = (words_.lower().split(" "))
            for word in words_:
                tokenized_marco_text.append(tokenize(word))

In [ ]:
with open("tokenized_marco/tokenized_marco_text","wb") as file:
    pickle.dump(tokenized_marco_text,file)

# with open("tokenized_marco/tokenized_marco_text","rb") as file:
#     pickle.load(file)  #To load the file

In [16]:
class embed_train_dataset(Dataset):
    def __init__(self, words, window=2):
        self.data = words
        self.window = window
    
    def __len__(self):
        return len(self.data)-4
    
    def __getitem__(self, idx):
        idx = idx+self.window    
        sent = self.data[max(0,idx-self.window):min(idx+self.window+1,len(self.data))]    
        if len(sent) > 1:
            rand_idx = random.randint(0,len(sent)-1)
            target = sent[rand_idx]
            del sent[rand_idx]
            #print (sent)
            tokenized = torch.tensor(sent)
            #print (tokenized)
            
            return tokenized, torch.tensor(target)
        



In [18]:
dataset = embed_train_dataset(tokenized_marco_text)
dataloader = DataLoader(dataset, batch_size=1,shuffle=True)

for data in dataloader:
    print (data)
    break



[tensor([[  679, 39993,   873,   168]]), tensor([1599])]
